In [16]:
#Import dependencies
import sqlite3
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [3]:
#Connect to DB
conn = sqlite3.connect("tweets.db")

In [4]:
with conn:
    cur = conn.cursor()
    #Construct SQL Queries
    #Count Total amount of Tweets in Database
    cur.execute("SELECT COUNT(USERID) FROM TWEETS")
    print("Total Tweets in DB: ", cur.fetchall()[0][0])
    #Count amount of manually labelled tweets currently in Database
    cur.execute("SELECT COUNT(USERID) FROM TWEETS WHERE ISHARASSMENT IS NOT NULL")
    print("Total Manual Labelled tweets in DB: ", cur.fetchall()[0][0])

Total Tweets in DB:  6480
Total Manual Labelled tweets in DB:  187


In [6]:
#Obtain all manually labelled tweets from DB
cur.execute("SELECT * FROM TWEETS WHERE ISHARASSMENT IS NOT NULL ORDER BY RANDOM()")
tweets = cur.fetchall()

In [7]:
tweets

[(1371598526254346247,
  '1258159387861975041',
  '@MouthPurdy FED chairman is the boy with his finger in the dyke. He is not going to pull that finger until at least 4% unemployment(10yrs). What’s behind the dyke is a years worth of unpaid mortgages another Fannie Freddie disaster. Landlords with nobody paying for 16 months.Please 2024 😂😂',
  1,
  0,
  0,
  0,
  0),
 (1372566486892617731,
  '71321856',
  '@Ronald69Swanson @ForTrannies @Davethe25219352 @Mag_Down_Under @laurenboebert I have a few questions about how you run Twitter trannie person, can you help with how I can stop ladies asking me if I was to join their fan only sites? I’m not into ladies Trannie person 😚',
  0,
  1,
  0,
  0,
  1),
 (1371957266576908288,
  '898738592746422272',
  '@victorcbueno Fuck that fag missed 6 throws',
  1,
  0,
  0,
  0,
  1),
 (1371582713954721795,
  '1354243124630085632',
  '“BESTIEE!!” i am not your friend fag.',
  1,
  0,
  0,
  0,
  1),
 (1365287109221883909,
  '744677802214064129',
  '@Pap

In [15]:
first = 0
for twid, uid, tweet, is_homophobic, is_transphobic, is_biphobic, has_pronouns, is_harassment in tweets:
    if first == 0:
        print(tweet)
        first = 1
    else:
        break

@MouthPurdy FED chairman is the boy with his finger in the dyke. He is not going to pull that finger until at least 4% unemployment(10yrs). What’s behind the dyke is a years worth of unpaid mortgages another Fannie Freddie disaster. Landlords with nobody paying for 16 months.Please 2024 😂😂


In [17]:
train, test = train_test_split(tweets, test_size=0.33, random_state=42)
train

[(1370852529639202821,
  '911339571333525506',
  '@straight_fag This is an absolute win wdym',
  1,
  0,
  0,
  0,
  0),
 (1365316390618615811,
  '1429407367',
  'If a dirt a dirt. Gimmie that batty. https://t.co/4amDXvxHTr',
  1,
  0,
  0,
  0,
  0),
 (1365278385249878017, '515856062', '3 day bender then?☀️', 1, 0, 0, 0, 0),
 (1365274260273582084,
  '20498725',
  'Just what I needed today. A batty happily munching a banana. \n\nhttps://t.co/nPU0JFyePI',
  1,
  0,
  0,
  None,
  0),
 (1365267112932032513,
  '1115633695660990464',
  '@OLUWA_DUROTIMI air bender. wagwan?',
  1,
  0,
  0,
  None,
  0),
 (1371606932319002628,
  '1182865032805261313',
  '@Emberflux not familiar with this, but one time i saw my friends on insta hanging out with the bru who called me a faggot (which they knew) and thag shit hurt',
  1,
  0,
  0,
  1,
  0),
 (1365275247042256901,
  '909850165770690560',
  'Lifestyle forecasts for Ponce Inlet Town Hall (https://t.co/pIEiC8xR7i) https://t.co/JT0dlDz4sc',
  0,
  0